Extract dataset

Powershell: `Get-ChildItem -Filter *.tar.gz | ForEach-Object {
    tar -xzf $_.FullName
    Remove-Item $_.FullName -Force
}`

Bash: `for file in *.tar.gz; do
    if [ -f "$file" ]; then
        tar xzvf "$file"
        rm "$file"
    fi
done`

In [ ]:
!nvidia-smi

In [2]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install rasterio

In [ ]:
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.config import get_cfg
from detectron2.data import detection_utils as utils
from detectron2.engine import DefaultTrainer
from detectron2.model_zoo import get_config_file, get_checkpoint_url
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data.datasets import register_coco_instances
import cv2
import os
import rasterio
import os
import glob
import rasterio
import json
from shapely.geometry import box, shape
from shapely.ops import transform
from pyproj import Proj, Transformer

import numpy as np



In [ ]:
def split_images_and_labels(image_path, label_path, output_dir, grid_size):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Use rasterio to read the image
    with rasterio.open(image_path) as src:
        image = src.read()
        height, width = src.height, src.width
        transform = src.transform
        crs = src.crs

    # Load COCO labels from GeoJSON
    with open(label_path, 'r') as f:
        geojson_data = json.load(f)

    # Parse COCO labels and convert to pixel coordinates
    bounding_boxes = []
    for feature in geojson_data['features']:
        properties = feature['properties']
        geometry = shape(feature['geometry'])

        cls = properties['category_id']
        if cls in boat_classes:
            # Convert geometry to pixel coordinates
            transformer = Transformer.from_crs(crs, "EPSG:4326", always_xy=True)
            geom_transformed = transform(transformer.transform, geometry)

            # Get bounding box in pixel coordinates
            minx, miny, maxx, maxy = geom_transformed.bounds
            bbox = box(minx, miny, maxx, maxy)
            x_center = (bbox.bounds[0] + bbox.bounds[2]) / 2 / width
            y_center = (bbox.bounds[1] + bbox.bounds[3]) / 2 / height
            bbox_width = (bbox.bounds[2] - bbox.bounds[0]) / width
            bbox_height = (bbox.bounds[3] - bbox.bounds[1]) / height

            bounding_boxes.append([class_mapping[cls], x_center, y_center, bbox_width, bbox_height])

    # Split the image into smaller patches
    rows, cols = grid_size
    patch_height, patch_width = height // rows, width // cols

    patch_counter = 0
    for i in range(rows):
        for j in range(cols):
            # Compute patch boundaries
            start_y, end_y = i * patch_height, (i + 1) * patch_height
            start_x, end_x = j * patch_width, (j + 1) * patch_width

            # Extract image patch
            patch = image[:, start_y:end_y, start_x:end_x]

            # Adjust and save the corresponding labels for the patch
            patch_labels = []
            contains_boat = False
            for box in bounding_boxes:
                cls, x_center, y_center, bbox_width, bbox_height = box

                # Convert to pixel coordinates
                x_center_abs = x_center * width
                y_center_abs = y_center * height
                bbox_width_abs = bbox_width * width
                bbox_height_abs = bbox_height * height

                # Check if bounding box is within the current patch
                if (start_x <= x_center_abs <= end_x) and (start_y <= y_center_abs <= end_y):
                    # Adjust bounding box for the new patch
                    new_x_center = (x_center_abs - start_x) / patch_width
                    new_y_center = (y_center_abs - start_y) / patch_height
                    new_bbox_width = bbox_width_abs / patch_width
                    new_bbox_height = bbox_height_abs / patch_height

                    patch_labels.append(f"{int(cls)} {new_x_center} {new_y_center} {new_bbox_width} {new_bbox_height}")
                    contains_boat = True

            # Save patch only if it contains boat classes
            if contains_boat:
                print(f"Saving patch {patch_counter} for image {os.path.basename(image_path)}")
                patch_filename = f"{os.path.splitext(os.path.basename(image_path))[0]}_{patch_counter}.tif"
                patch_path = f"{output_dir}/{patch_filename}"
                with rasterio.open(
                    patch_path,
                    'w',
                    driver='GTiff',
                    height=patch_height,
                    width=patch_width,
                    count=image.shape[0],
                    dtype=image.dtype,
                ) as dst:
                    dst.write(patch)

                # Save labels for this patch
                patch_label_filename = f"{os.path.splitext(os.path.basename(label_path))[0]}_{patch_counter}.txt"
                patch_label_path = f"{output_dir}/{patch_label_filename}"

                with open(patch_label_path, 'w') as f:
                    for label in patch_labels:
                        f.write(label + '\n')

            patch_counter += 1

# Split all images and labels in the directory
all_tifs = glob.glob(f"/content/drive/MyDrive/ShipClassification/EO_Train/*.tif")
for tif in all_tifs:
    label_path = tif.replace(".tif", ".geojson")
    split_images_and_labels(tif, label_path, "/content/drive/MyDrive/ShipClassification/EO_Train_Split3", (8, 8))


In [ ]:
def process_scene(scene_dir, processed_dir):
    # Add stacked image to processed_dir
    scene_id = os.path.basename(scene_dir)
    combine_sar_channels(f"{scene_dir}/VV_dB.tif", f"{scene_dir}/VH_dB.tif", f"{processed_dir}/{scene_id}_stacked.tif")

    # Add label file to processed_dir
    convert_csv_to_yolo_labels(f"{scene_dir}/labels.csv", processed_dir, scene_dir)

    # Split image



In [4]:
# combine VV and VH into a single grayscale channel (if time, compare to split channels)
def custom_read_image(file_name, format=None):
    import rasterio
    import numpy as np

    with rasterio.open(file_name) as src:
        vv = src.read(1)  # VV polarization
        vh = src.read(2)  # VH polarization

        # combine into single grayscale ch.
        grayscale = (vv + vh) / 2

        # Duplicate ch to work with rgb transfer model
        image = np.stack([vv, vh, grayscale], axis=-1)  # (H, W, 3)
        return image

utils.read_image = custom_read_image
config_path = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"

In [24]:
# Register the dataset
register_coco_instances("cascade_train_dataset", {}, "train_coco.json", "/content/drive/MyDrive/ShipClassfication/SAR/Train")
register_coco_instances("cascade_val_dataset", {}, "validation_coco.json", "/content/drive/MyDrive/ShipClassfication/SAR/Val")
# Set up the configuration
cfg = get_cfg()
cfg.merge_from_file(get_config_file(config_path))  # Use pre-configured Cascade R-CNN
cfg.MODEL.PIXEL_MEAN = [123.0, 123.0, 123.0]  # Currently set to three channels for flattened pols
cfg.MODEL.PIXEL_STD = [58.0, 58.0, 58.0]     #
cfg.DATASETS.TRAIN = ("cascade_train_dataset",)
cfg.DATASETS.TEST = ("cascade_val_dataset",)
cfg.DATALOADER.NUM_WORKERS = 0
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/ShipClassfication/Checkpoints/Faster_RCNN_1.pth"
cfg.SOLVER.IMS_PER_BATCH = 1
cfg.SOLVER.BASE_LR = 0.01
cfg.SOLVER.MAX_ITER = 100
# cfg.SOLVER.STEPS = [300, 400]
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
cfg.SOLVER.CHECKPOINT_PERIOD = 50
cfg.SOLVER.AMP.ENABLED = True
cfg.MODEL.DEVICE = "cuda"


In [ ]:
generation = 7
def main():
    # Output directory for saving checkpoints and logs
    cfg.OUTPUT_DIR = f"/content/drive/MyDrive/ShipClassfication/Checkpoints/SAR/{generation}"

    cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[8, 16, 32, 64, 128]]  # Smaller anchor sizes
    cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.75, 1.0, 1.25]]  # Use default aspect ratios

    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

    # Train the model
    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=True)
    trainer.train()




if __name__ == "__main__":
    main()

In [ ]:
   # Create evaluator and test loader
    evaluator = COCOEvaluator("cascade_val_dataset", cfg, False, output_dir=f"./output_cascade_rcnn{generation}")
    test_loader = build_detection_test_loader(cfg, "cascade_val_dataset")
    metrics = inference_on_dataset(trainer.model, test_loader, evaluator)

    predictions  = []
    ground_truth = []
    # Perform evaluation
    inference_on_dataset(trainer.model, test_loader, evaluator)

    # Load the trained model
    # cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
    cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/ShipClassfication/Checkpoints/Faster_RCNN_1.pth"
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set a threshold for predictions
    predictor = DefaultPredictor(cfg)

    # Load and predict on a new image
    image_path = "/content/drive/MyDrive/ShipClassfication/SAR/Val/72dba3e82f782f67t_stacked.tif"
    image = custom_read_image(image_path)
    outputs = predictor(image)



In [ ]:
# Load model
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/ShipClassfication/Checkpoints/Faster_RCNN_1.pth"
predictor = DefaultPredictor(cfg)

# Create evaluator and test loader
evaluator = COCOEvaluator("cascade_val_dataset", cfg, False, output_dir="/content/drive/MyDrive/ShipClassfication/Checkpoints/SAR/7")
test_loader = build_detection_test_loader(cfg, "cascade_val_dataset")

# Perform evaluation
metrics = inference_on_dataset(predictor.model, test_loader, evaluator)

precision = metrics["bbox"]["AP"]
recall = metrics["bbox"]["AR"]
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1_score}")

In [ ]:
   # Visualize model on test data

    v = Visualizer(image[:, :, ::-1], MetadataCatalog.get("my_train_dataset"), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    meta = image.meta.copy()
    meta.update({
        'count': 3,
        'dtype': image.read(1).dtype,
    })

    with rasterio.open(f"{generation}_ouput.tif", 'w', **meta) as dst:
        dst.write(image)